# <span style="color:green">Formation South Green 2022</span> - Structural Variants Detection by using short and long reads 

# __DAY 1 : How to map reads against a reference genome ?__ 

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

## __1. Preparing the working environment__ 

### First create a dedicated folder to work 


In [ ]:
# go to work directory and download data
cd /home/jovyan/work/
ls

## Download sequencing data (SR & LR) for Simulated clones

Before starting, please download special data created for this practical training. Data are available on the from I-Trop server.

Each participant will analyse a Clone, results will be complete in this shared file

To generate Clone data, a 1Mb contig was extracted from chromosome 1 of rice.

20 levels of variation were generated and long reads were simulated for each.

We have introduced different variations (SNP, indel, indel+translocations) and also some contaminations.

In [ ]:
# download available compressed DATA 
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/sv-training/SV_DATA.tar.gz
# decompress data
tar zxvf SV_DATA.tar.gz
rm SV_DATA.tar.gz

In [ ]:
# check data 
ls -l SV_DATA/

-----------------------
# 2. MAPPING PRACTICE

Read congruency is an important measure in determining assembly accuracy.

Clusters of read pairs or single long reads that align incorrectly are strong indicators of mis-assembly.

Reads mapping is usually the firt step before SNP or Variant calling.

### 2.1 Make a folder for your results

In [ ]:
mkdir -p ~/work/MAPPING-ILL
cd ~/work/MAPPING-ILL

### 2.2 Declare important variables

In [ ]:
i=10
REF_DIR="/home/jovyan/work/SV_DATA/REF/"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
ONT="/home/jovyan/work/SV_DATA/LONG_READS/Clone${i}.fastq.gz"
ILL_R1="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R1.fastq.gz"
ILL_R2="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R2.fastq.gz"

In [ ]:
echo "Clone${i} $REF" 
echo $ILL_R1 $ILL_R2

## 2.1 Mapping short reads vs a reference with `bwa mem`

In this practice, we are going to map short reads against a reference. To know, how well do the reads align back to the reference, we use bwa-mem2 and samtools to assess the basic alignment statistics.

In this exercise, we will use reference.fasta assembly as well ILLUMINA READS from your favorite CLONE.

The tool bwa needs 2 steps: 
- **Reference indexing**: `bwa index reference`
- **Mapping in itself**: `bwa mem  -R READGROUP [options] reference fastq1 fastq2 > out.sam`

## Reference indexation 

Before mapping we need index reference file! Check bwa-mem2 index command line.

In [ ]:
cd $REF_DIR

In [ ]:
echo -e "\nIndexing reference $REF\n"
bwa-mem2 index $REF

In [ ]:
pwd

In [ ]:
ls

##  Let's map now but only WITH READS FROM ONLY ONE CLONE

In [ ]:
cd ~/work/MAPPING-ILL
echo -e "\nCreation directory for Clone$i\n"
echo Clone$i
mkdir -p dirClone$i
cd dirClone$i

In [ ]:
echo -e "\nMapping Clone$i\n"
bwa-mem2 mem -M -t 4 $REF $ILL_R1 $ILL_R2 > Clone$i.sam

In [ ]:
pwd 

In [ ]:
ls

In [ ]:
head Clone10.sam

##  Calculate stats from mapping

In [ ]:
samtools view -@4 -bh -S -o  Clone$i.bam Clone$i.sam 
rm Clone$i.sam
samtools flagstat Clone$i.bam >Clone$i.flagstats

##  Calculate stats only for correctly mapped reads

In [ ]:
samtools view -bh -@4 -f 0x02 -o Clone$i.mappedpaired.bam Clone$i.bam 
samtools flagstat Clone$i.mappedpaired.bam > Clone$i.mappedpaired.flagstats

##  Sorting final bam

In [ ]:
samtools sort -@4 Clone$i.mappedpaired.bam Clone$i.SORTED 
rm Clone$i.mappedpaired.bam

##  Indexing bam

In [ ]:
samtools index Clone$i.SORTED.bam

In [ ]:
ls -lrt

## Let's map with data from all clones using a loop for mapping, with a single folder per sample

In [ ]:
for i in {1..20}
    do
        cd ~/work/MAPPING-ILL
        echo -e "\nCreation directory for Clone$i\n"
        echo Clone$i
        mkdir -p dirClone$i
        cd dirClone$i
        
        echo -e "\nDeclare variables$i\n"
        REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
        ILL_R1="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R1.fastq.gz"
        ILL_R2="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R2.fastq.gz"

        echo -e "\nMapping Clone$i\n"
        bwa-mem2 mem -M -t 4 $REF $ILL_R1 $ILL_R2 > Clone$i.sam
        
        echo -e "\nMapping Clone$i\n"
        samtools view -@4 -bh -S -o  Clone$i.bam Clone$i.sam 
        rm Clone$i.sam
        echo -e "\nFlagstats from all reads $i\n"
        samtools flagstat Clone$i.bam >Clone$i.flagstats
        
        echo -e "\nExtract only correctly mapped and calculate flagstats $i\n"
        samtools view -bh -@4 -f 0x02 -o Clone$i.mappedpaired.bam Clone$i.bam 
        samtools flagstat Clone$i.mappedpaired.bam > Clone$i.mappedpaired.flagstats
        
        echo -e "\nSort mappedpaires bam file $i\n"
        samtools sort -@4 Clone$i.mappedpaired.bam Clone$i.SORTED 
        rm Clone$i.mappedpaired.bam
    done

## 2.2 Mapping Long reads vs a Reference

Similar process such as SR is done in LR. In this case mapper is minimap2.

In [ ]:
# Declare variables
i=10
REF_DIR="/home/jovyan/work/SV_DATA/REF/"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
ONT="/home/jovyan/work/SV_DATA/LONG_READS/Clone${i}.fastq.gz"

##  Let's map now but only WITH READS FROM ONLY ONE CLONE

In [ ]:
mkdir -p ~/work/MAPPING-ONT
cd ~/work/MAPPING-ONT
echo -e "\nCreation directory for Clone$i\n"
echo Clone$i
mkdir -p dirClone$i
cd dirClone$i

In [ ]:
echo -e "\nMapping Clone$i minimap2 \n"
minimap2 -ax map-ont -t 4 ${REF} ${ONT} > Clone${i}_ONT.sam 

In [ ]:
head -n3 Clone${i}_ONT.sam

In [ ]:
## Convert samtobam 
echo -e "\nConvert samtobam and filter it \n"
samtools view -@4 -bh -S -F 0x904 -o Clone${i}_ONT.bam Clone${i}_ONT.sam
rm Clone${i}_ONT.sam

In [ ]:
echo -e "\nSort and index bam \n"
# sort and index bam
samtools sort -@4 Clone${i}_ONT.bam Clone${i}_ONT_SORTED 
samtools index Clone${i}_ONT_SORTED.bam

In [ ]:
# Calculate stats from mapping
echo -e "\nCalculate stats from mapping\n"
samtools flagstat Clone${i}_ONT_SORTED.bam >Clone${i}_ONT.flagstats

## Let's map with data from all clones using a loop for mapping, with a single folder per sample and ONT reads

In [ ]:
for i in {1..20}
    do
        mkdir -p ~/work/MAPPING-ONT
        cd ~/work/MAPPING-ONT
        echo -e "\nCreation directory for Clone$i\n"
        echo Clone$i
        mkdir -p dirClone$i
        cd dirClone$i
        
        echo -e "\nMapping Clone$i minimap2 \n"
        minimap2 -ax map-ont -t 4 ${REF} ${ONT} > Clone${i}_ONT.sam 
        
        # Convert samtobam 
        echo -e "\nConvert samtobam and filter it \n"
        samtools view -@4 -bh -S -F 0x904 -o Clone${i}_ONT.bam Clone${i}_ONT.sam
        rm Clone${i}_ONT.sam

        echo -e "\nSort and index bam \n"
        # sort and index bam
        samtools sort -@4 Clone${i}_ONT.bam Clone${i}_ONT_SORTED 
        samtools index Clone${i}_ONT_SORTED.bam

        # Calculate stats from mapping
        echo -e "\nCalculate stats from mapping\n"
        samtools flagstat Clone${i}_ONT_SORTED.bam >Clone${i}_ONT.flagstats
    done

In [ ]:
ls

# REORDER BAM FILES INTO A FOLDER ONLY TO ILLUMINA

In [ ]:
mkdir -p ~/work/MAPPING-ILL/BAM
cd ~/work/MAPPING-ILL/

for i in {1..20}
    do
         ln -s ~/work/MAPPING-ILL/dirClone$i/Clone${i}.bam BAM/
    done

# REORDER BAM FILES INTO A FOLDER NOW FOR ONT